Lambda School Data Science

*Unit 3, Med Cabinet Build*

---

In [18]:
import pandas as pd
import numpy as np


pd.set_option('display.max_rows', 500)
df = pd.read_csv('cannabis.csv')
df = df.replace('None', np.NaN)
df = df.replace('Dry', np.NaN)
df = df.dropna()


df.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Floral,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy,Herbal,Sage,Wood",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [19]:
# df.loc[df['SEARCH_HERE'].isnull() == True]

df.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         0
Description    0
dtype: int64

## Step 1 - Effects/Flavor Search Preparation

We need to break out the effects for each strain into something searchable. To do this we're going to grab the number of unique entries, this way we can know how any new columns we'll have for encoding.

In [20]:
# This is what one row looks like on efects

print(df['Effects'][0])

# Python considers this a string

print(type(df['Effects'][0]))

Creative,Energetic,Tingly,Euphoric,Relaxed
<class 'str'>


In [21]:
# Pandas has an option to turn strings in a series into lists through the split method.
# Since methods run across the whole series we need to tell it to focus on the strings for each and split that,
# otherwise it thinks we're trying to split the series, which makes no sense.

df['Effects_List'] = df['Effects'].str.split(',')
df['Flavor_List']  = df['Flavor'].str.split(',')

df['Effects_List']

0       [Creative, Energetic, Tingly, Euphoric, Relaxed]
1         [Relaxed, Aroused, Creative, Happy, Energetic]
2        [Uplifted, Happy, Relaxed, Energetic, Creative]
3          [Tingly, Creative, Hungry, Relaxed, Uplifted]
4        [Happy, Relaxed, Euphoric, Uplifted, Talkative]
                              ...                       
2346     [Happy, Uplifted, Relaxed, Euphoric, Energetic]
2347        [Relaxed, Happy, Euphoric, Uplifted, Sleepy]
2348       [Relaxed, Sleepy, Talkative, Euphoric, Happy]
2349          [Relaxed, Sleepy, Euphoric, Happy, Hungry]
2350          [Hungry, Relaxed, Uplifted, Happy, Sleepy]
Name: Effects_List, Length: 2187, dtype: object

In [22]:
# Now python sees the field as a list.

print(type(df['Effects_List'][0]))
print(df['Effects_List'][0])

<class 'list'>
['Creative', 'Energetic', 'Tingly', 'Euphoric', 'Relaxed']


In [23]:
# From here we can see that while some are below 5, none surpass it.
# We might need to do something about that, but for now we can ignore it.

df['Effects_List'].str.len()

0       5
1       5
2       5
3       5
4       5
       ..
2346    5
2347    5
2348    5
2349    5
2350    5
Name: Effects_List, Length: 2187, dtype: int64

In [27]:
# So we can see we have 15 unique values. Now we need to encode this.

print(len(df['Effects_List'].apply(pd.Series).stack().value_counts())
     ,df['Effects_List'].apply(pd.Series).stack().value_counts()
     ,len(df['Flavor_List'].apply(pd.Series).stack().value_counts())
     ,df['Flavor_List'].apply(pd.Series).stack().value_counts()
     )

13 Happy        1821
Relaxed      1682
Euphoric     1594
Uplifted     1468
Creative      722
Sleepy        721
Energetic     620
Focused       581
Hungry        459
Talkative     348
Tingly        331
Giggly        277
Aroused       191
dtype: int64 53 Earthy        1096
Sweet         1047
Citrus         521
Pungent        445
Berry          353
Pine           298
Wood           285
Floral         267
Diesel         240
Herbal         227
Spicy          227
Lemon          190
Skunk          169
Blueberry      153
Tropical       153
Grape          127
Orange          76
Cheese          67
Pepper          59
Lime            52
Strawberry      47
Minty           41
Pineapple       41
Sage            39
Grapefruit      38
Chemical        38
Lavender        37
Vanilla         34
Mango           33
Fruity          33
Honey           32
Ammonia         28
Nutty           24
Coffee          24
Menthol         22
Butter          19
Mint            18
Tea             18
Apple           16
Rose  

In [28]:
df['Flavor_List'].apply(pd.Series).stack().value_counts()

Earthy        1096
Sweet         1047
Citrus         521
Pungent        445
Berry          353
Pine           298
Wood           285
Floral         267
Diesel         240
Herbal         227
Spicy          227
Lemon          190
Skunk          169
Blueberry      153
Tropical       153
Grape          127
Orange          76
Cheese          67
Pepper          59
Lime            52
Strawberry      47
Minty           41
Pineapple       41
Sage            39
Grapefruit      38
Chemical        38
Lavender        37
Vanilla         34
Mango           33
Fruity          33
Honey           32
Ammonia         28
Nutty           24
Coffee          24
Menthol         22
Butter          19
Mint            18
Tea             18
Apple           16
Rose            16
Tobacco          8
Apricot          8
Tar              7
Violet           7
Chestnut         6
Peach            5
Sour             4
Pear             3
Plum             2
Melon            1
Tart             1
Candy            1
Tangy       

## Step 2 - Encoding

In [29]:
# These two do the same thing, but they don't work with lists.
# They also only work with EXACT MATCHES.

df.loc[df['Effects'] == 'Creative']

df.loc[df['Effects'].isin(['Creative'])]

,Strain,Type,Rating,Effects,Flavor,Description,Effects_List,Flavor_List
369,Brain-Candy,hybrid,5.0,Creative,Sweet,Brain Candy by Insanity Strains is a handy hyb...,[Creative],[Sweet]


### MultiLabelBinarizer

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

print(
pd.DataFrame(mlb.fit_transform(df['Effects_List'])
            ,columns = mlb.classes_
            ,index   = df.index
            )
     )

      Aroused  Creative  Energetic  Euphoric  Focused  Giggly  Happy  Hungry  \
0           0         1          1         1        0       0      0       0   
1           1         1          1         0        0       0      1       0   
2           0         1          1         0        0       0      1       0   
3           0         1          0         0        0       0      0       1   
4           0         0          0         1        0       0      1       0   
...       ...       ...        ...       ...      ...     ...    ...     ...   
2346        0         0          1         1        0       0      1       0   
2347        0         0          0         1        0       0      1       0   
2348        0         0          0         1        0       0      1       0   
2349        0         0          0         1        0       0      1       1   
2350        0         0          0         0        0       0      1       1   

      Relaxed  Sleepy  Talkative  Tingl

In [31]:
mlb_effects_df = pd.DataFrame(mlb.fit_transform(df['Effects_List'])
                              ,columns = mlb.classes_
                              ,index   = df.index
                              )

mlb_flavor_df = pd.DataFrame(mlb.fit_transform(df['Flavor_List'])
                            ,columns = mlb.classes_
                            ,index   = df.index
                            )
print(mlb_effects_df.head()
     ,mlb_flavor_df.head()
     )

   Aroused  Creative  Energetic  Euphoric  Focused  Giggly  Happy  Hungry  \
0        0         1          1         1        0       0      0       0   
1        1         1          1         0        0       0      1       0   
2        0         1          1         0        0       0      1       0   
3        0         1          0         0        0       0      0       1   
4        0         0          0         1        0       0      1       0   

   Relaxed  Sleepy  Talkative  Tingly  Uplifted  
0        1       0          0       1         0  
1        1       0          0       0         0  
2        1       0          0       0         1  
3        1       0          0       1         1  
4        1       0          1       0         1      Ammonia  Apple  Apricot  Berry  Blueberry  Butter  Candy  Cheese  Chemical  \
0        0      0        0      0          0       0      0       0         0   
1        0      0        0      0          0       0      0       0        

## Step 3 - Merge

In [32]:
mlb_effects_df = mlb_effects_df.merge(mlb_flavor_df
                                     ,left_index = True
                                     ,right_index = True
                                     )
mlb_df         = df.merge(mlb_effects_df
                         ,left_index = True
                         ,right_index = True
                         )

mlb_df         = mlb_df.drop(['Effects_List', 'Flavor_List'], axis = 1)

In [33]:
mlb_df.columns

Index(['Strain', 'Type', 'Rating', 'Effects', 'Flavor', 'Description',
       'Aroused', 'Creative', 'Energetic', 'Euphoric', 'Focused', 'Giggly',
       'Happy', 'Hungry', 'Relaxed', 'Sleepy', 'Talkative', 'Tingly',
       'Uplifted', 'Ammonia', 'Apple', 'Apricot', 'Berry', 'Blueberry',
       'Butter', 'Candy', 'Cheese', 'Chemical', 'Chestnut', 'Citrus', 'Coffee',
       'Diesel', 'Earthy', 'Floral', 'Fruity', 'Grape', 'Grapefruit', 'Herbal',
       'Honey', 'Lavender', 'Lemon', 'Lime', 'Mango', 'Melon', 'Menthol',
       'Mint', 'Minty', 'Nutty', 'Orange', 'Peach', 'Pear', 'Pepper', 'Pine',
       'Pineapple', 'Plum', 'Pungent', 'Rose', 'Sage', 'Skunk', 'Sour',
       'Spicy', 'Strawberry', 'Sweet', 'Tangy', 'Tar', 'Tart', 'Tea',
       'Tobacco', 'Tropical', 'Vanilla', 'Violet', 'Wood'],
      dtype='object')

## Step 4 - SQL

In [53]:
import sqlite3
import SQL


mlb_df.to_sql('cannabis'
             ,con       = sqlite3.Connection('cannabis.sqlite3')
             ,if_exists = 'replace'
             )

sql_df     = SQL.SQL('cannabis.sqlite3')

In [79]:
df.head()

,Strain,Type,Rating,Effects,Flavor,Description,Effects_List,Flavor_List
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"[Creative, Energetic, Tingly, Euphoric, Relaxed]","[Earthy, Sweet, Citrus]"
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Floral,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"[Relaxed, Aroused, Creative, Happy, Energetic]","[Floral, Violet, Diesel]"
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy,Herbal,Sage,Wood",1024 is a sativa-dominant hybrid bred in Spain...,"[Uplifted, Happy, Relaxed, Energetic, Creative]","[Spicy, Herbal, Sage, Wood]"
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"[Tingly, Creative, Hungry, Relaxed, Uplifted]","[Apricot, Citrus, Grapefruit]"
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","[Happy, Relaxed, Euphoric, Uplifted, Talkative]","[Citrus, Earthy, Orange]"


In [74]:
for column in mlb_df.columns:
    print(column)

Strain
Type
Rating
Effects
Flavor
Description
Aroused
Creative
Energetic
Euphoric
Focused
Giggly
Happy
Hungry
Relaxed
Sleepy
Talkative
Tingly
Uplifted
Ammonia
Apple
Apricot
Berry
Blueberry
Butter
Candy
Cheese
Chemical
Chestnut
Citrus
Coffee
Diesel
Earthy
Floral
Fruity
Grape
Grapefruit
Herbal
Honey
Lavender
Lemon
Lime
Mango
Melon
Menthol
Mint
Minty
Nutty
Orange
Peach
Pear
Pepper
Pine
Pineapple
Plum
Pungent
Rose
Sage
Skunk
Sour
Spicy
Strawberry
Sweet
Tangy
Tar
Tart
Tea
Tobacco
Tropical
Vanilla
Violet
Wood


In [80]:
# sql_df.query('SELECT Effects FROM cannabis')